In [1]:
#from dataclasses import dataclass
import numpy as np
from plotly.io import show
import plotly.io as pio
pio.renderers.default = 'colab'
import plotly.express as px
import pyproximal
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from skfolio import Population, RiskMeasure, Portfolio
from skfolio.datasets import load_sp500_dataset, load_nasdaq_dataset, load_ftse100_dataset
from skfolio.optimization import EqualWeighted, MeanRisk, ObjectiveFunction
from skfolio.preprocessing import prices_to_returns

In [2]:
from algorithms.base import Trainer, Portfolio_Optimization, RISK_TYPE
from algorithms.utility_based_shortfall_risk import UBSR
from algorithms.optimized_certainty_equivalent import OCE, OCE_Risk
from utils import unit_projection

# Market Data Analysis 

In [28]:
prices = load_ftse100_dataset()
X = prices_to_returns(prices)
X_train, X_test = train_test_split(X, test_size=0.5, shuffle=False)
mean_return = X_train.mean()
R, d = X.shape
R, d

(5959, 64)

# UBSR-SG Algorithm Implementation

## Portfolio 1 : Min CVaR portfolio

In [29]:
cvar_model = MeanRisk(
    risk_measure=RiskMeasure.CVAR,
    objective_function=ObjectiveFunction.MINIMIZE_RISK,
    portfolio_params=dict(name="Min CVaR"),
)
cvar_model.fit(X_train)

MeanRisk(portfolio_params={'name': 'Min CVaR'}, risk_measure=CVaR)

## Portfolio 2 : Equal weighted portfolio

In [30]:
benchmark = EqualWeighted(portfolio_params=dict(name="Equal Weighted"))
benchmark.fit(X_train)

EqualWeighted(portfolio_params={'name': 'Equal Weighted'})

In [31]:
EPOCHS = 20000
projection = pyproximal.projection.SimplexProj(d, radius=1, maxiter = 2000, xtol = 1e-6)
my_projection = (lambda x :unit_projection(projection(x), np.ceil(np.log10(d)).astype(int)))

## Portfolio 3: UBSR (Entropic)
$$l(x) = e^{\beta x}, \beta > 0$$

In [32]:
BETA = 0.5
l = (lambda x : np.exp(BETA * x))
threshold = 1
l_prime, delta, epsilon = (lambda x : BETA*np.exp(BETA * x)), 1e-3, 1.
sr = UBSR('Entropic risk', RISK_TYPE.UBSR, l, l_prime, delta, epsilon, Portfolio_Optimization, my_projection)
entropic_risk = Trainer(sr, EPOCHS, -5, 5) 
entropic_risk.get_figure().show()

## Portfolio 4: UBSR (Expectile Risk)
$$l(x) = \alpha x^+ - (1-\alpha)x^-, \alpha \in (1/2,1)$$

In [33]:
ALPHA = 0.66
l, threshold = (lambda x : np.where(x>0, ALPHA * x, (1- ALPHA)*x)),  0.
l_prime, delta, epsilon = (lambda x : np.where(x>0, ALPHA, (1- ALPHA))), 1e-3, 1.
sr = UBSR('Expectile risk', RISK_TYPE.UBSR, l, l_prime, delta, epsilon, Portfolio_Optimization, my_projection)
expectile_risk = Trainer(sr, EPOCHS, -5, 5) 
expectile_risk.get_figure().show()

#### Portfolio 5: UBSR (Arctan Risk I)
$$ l(x) = x - \frac{2}{\pi}tan^{-1}(x)$$

In [34]:
l = (lambda x : x - 2*(np.arctan(x) / np.pi) )
threshold = 0
l_prime, delta, epsilon = (lambda x : 1 -  (2 / (np.pi*(1+x**2)))) , 1e-3, 1.
sr = UBSR('Arctan risk I', RISK_TYPE.UBSR, l, l_prime, delta, epsilon, Portfolio_Optimization, my_projection)
arctan_risk_1 = Trainer(sr, EPOCHS, -5, 5) 
arctan_risk_1.get_figure().show()

#### Portfolio 6: UBSR (Sine Risk)
$$ l(x) = \sqrt{2} x - sin(x)$$

In [35]:
l = (lambda x : x - np.sin(x))
threshold = l(mean_return)+1e-6
l_prime, delta, epsilon = (lambda x : np.sqrt(2) - np.cos(x)) , 1e-3, 1.
sr = UBSR('Sine risk', RISK_TYPE.UBSR, l, l_prime, delta, epsilon, Portfolio_Optimization, my_projection)
sine_risk = Trainer(sr, EPOCHS, -5, 5) 
sine_risk.get_figure().show()

#### Portfolio 7: UBSR (Arctan Risk II)
$$ l(x) = \frac{2 x^2 tan^{-1}(x)}{\pi} $$ 

In [36]:
l = (lambda x : (x**2) * 2 * np.arctan(x) / np.pi)
threshold = l(mean_return)+1e-6
l_prime, delta, epsilon = (lambda x : (2 / np.pi) * (2 * x * np.arctan(x) +  (x**2 / (1+x**2)))) , 1e-3, 1.
sr = UBSR('Arctan risk II', RISK_TYPE.UBSR, l, l_prime, delta, epsilon, Portfolio_Optimization, my_projection)
arctan_risk_2 = Trainer(sr, EPOCHS, -5, 5) 
arctan_risk_2.get_figure().show()

#### Portfolio 8: UBSR (Tanh risk)
$$ l(x) = x^2 tanh(x) $$

In [37]:
l = (lambda x : (x**2) * np.tanh(x))
threshold = l(mean_return)+1e-6
l_prime, delta, epsilon = (lambda x : 2 * x * np.tanh(x) +  (x**2 / (np.cosh(x)**2))) , 1e-3, 1.
sr = UBSR('Tanh Risk', RISK_TYPE.UBSR, l, l_prime, delta, epsilon, Portfolio_Optimization, my_projection)
tanh_risk = Trainer(sr, EPOCHS, -5, 5) 
tanh_risk.get_figure().show()

## Portfolio 9: Averse Risk

In [38]:
l = (lambda x : np.where(x>0, (x**2) * np.tanh(x), -np.log(1+x**2)))
threshold = 0 #l(mean_return)
l_prime, delta, epsilon = (lambda x : np.where(x>0, 2 * x * np.tanh(x) +  (x**2 / (np.cosh(x)**2)),(-2*x)/(1+x**2))) , 1e-3, 1.
sr = UBSR('Averse Risk', RISK_TYPE.UBSR, l, l_prime, delta, epsilon, Portfolio_Optimization, my_projection)
averse_risk = Trainer(sr, EPOCHS, -5, 5) 
averse_risk.get_figure().show()

## Training the risk models

In [39]:
# for risk in [entropic_risk, expectile_risk, arctan_risk_1, sine_risk, arctan_risk_2, tanh_risk, averse_risk]:
#     risk.train(X_train)
for risk in [entropic_risk, expectile_risk, averse_risk]:
    risk.train(X_train)

100%|██████████| 20000/20000 [02:17<00:00, 145.64it/s]


## Backtesting results: Full data

In [61]:
population = Population([
    Portfolio(X=X_train, weights=cvar_model.weights_, name='min CVaR'),
    Portfolio(X=X_train, weights=benchmark.weights_, name='Equal weights'),
    Portfolio(X=X_train, weights=entropic_risk.weights_, name=entropic_risk.risk.name),
    Portfolio(X=X_train, weights=expectile_risk.weights_, name=expectile_risk.risk.name),
    #Portfolio(X=X_train, weights=arctan_risk_1.weights_, name=arctan_risk_1.risk.name),
    #Portfolio(X=X_train, weights=sine_risk.weights_, name=sine_risk.risk.name),
    #Portfolio(X=X_train, weights=arctan_risk_2.weights_, name=arctan_risk_2.risk.name),
    #Portfolio(X=X_train, weights=tanh_risk.weights_, name=tanh_risk.risk.name),
    Portfolio(X=X_train, weights=averse_risk.weights_, name=averse_risk.risk.name)
])
population.plot_composition()

In [62]:
fig = population.plot_cumulative_returns()
show(fig)

#### Backtesting results: Test data

In [63]:
population = Population([
    Portfolio(X=X_test, weights=cvar_model.weights_, name='min CVaR'),
    Portfolio(X=X_test, weights=benchmark.weights_, name='Equal weights'),
    Portfolio(X=X_test, weights=entropic_risk.weights_, name=entropic_risk.risk.name),
    Portfolio(X=X_test, weights=expectile_risk.weights_, name=expectile_risk.risk.name),
    #Portfolio(X=X_test, weights=arctan_risk_1.weights_, name=arctan_risk_1.risk.name),
    #Portfolio(X=X_test, weights=sine_risk.weights_, name=sine_risk.risk.name),
    #Portfolio(X=X_test, weights=arctan_risk_2.weights_, name=arctan_risk_2.risk.name),
    #Portfolio(X=X_test, weights=tanh_risk.weights_, name=tanh_risk.risk.name),
    Portfolio(X=X_test, weights=averse_risk.weights_, name=averse_risk.risk.name)
])
population.plot_composition()

In [64]:
fig = population.plot_cumulative_returns()
show(fig)

# OCE-SG Algorithm Implementation

## OCE (Entropic Risk) 

In [44]:
BETA = 0.5
u, u_prime = (lambda x : np.exp(BETA * x)), (lambda x : BETA * np.exp(BETA * x))  
delta, epsilon = 1e-3, 1.
oce = OCE_Risk('Entropic Risk (OCE)', RISK_TYPE.OCE, u, u_prime, delta, epsilon, Portfolio_Optimization, my_projection)
entropic_risk_oce = Trainer(oce, EPOCHS, -5, 5) 
entropic_risk_oce.get_figure().show()

## OCE (CVaR Risk)

In [45]:
ALPHA = 0.1
u, u_prime = (lambda x : np.where(x>0, x, 0)/(1-ALPHA)), (lambda x : np.where(x>0, 1, 0)/(1-ALPHA))
delta, epsilon = 1e-3, 1.
oce = OCE_Risk('CVaR Risk', RISK_TYPE.OCE, u, u_prime, delta, epsilon, Portfolio_Optimization, my_projection)
cvar_risk = Trainer(oce, EPOCHS, -5, 5) 
cvar_risk.get_figure().show()

## OCE (Monotone Mean-Variance Risk)

In [46]:
ALPHA = 2
u, u_prime = (lambda x : (np.maximum(1+x,0)**ALPHA - 1)/ALPHA), (lambda x : np.maximum(1+x,0)**(ALPHA-1))
delta, epsilon = 1e-3, 1.
oce = OCE_Risk('Mono-MV Risk', RISK_TYPE.OCE, u, u_prime, delta, epsilon, Portfolio_Optimization, my_projection)
mono_mv_risk = Trainer(oce, EPOCHS, -5, 5) 
mono_mv_risk.get_figure().show()

## OCE (Quartic Utility)

In [47]:
u = lambda x : np.where(x>-1, (-1-x)**4-1, -1)
u_prime = lambda x : np.where(x >- 1, -4*(-1-x)**3 , 0)
delta, epsilon = 1e-3, 1.
oce = OCE_Risk('Quartic Risk', RISK_TYPE.OCE, u, u_prime, delta, epsilon, Portfolio_Optimization, my_projection)
quartic_risk = Trainer(oce, EPOCHS, -5, 5) 
quartic_risk.get_figure().show()

In [48]:
for risk in [entropic_risk_oce, mono_mv_risk, quartic_risk]:
    risk.train(X_train)

100%|██████████| 20000/20000 [11:55<00:00, 27.95it/s]


In [57]:
population = Population([
    Portfolio(X=X_train, weights=cvar_model.weights_, name='min CVaR'),
    Portfolio(X=X_train, weights=benchmark.weights_, name='Equal weights'),
    Portfolio(X=X_train, weights=entropic_risk_oce.weights_, name=entropic_risk_oce.risk.name),
    #Portfolio(X=X_train, weights=cvar_risk.weights_, name=cvar_risk.name),
    Portfolio(X=X_train, weights=mono_mv_risk.weights_, name=mono_mv_risk.risk.name),
    Portfolio(X=X_train, weights=quartic_risk.weights_, name=quartic_risk.risk.name)
    ])
population.plot_composition()

In [58]:
fig = population.plot_cumulative_returns()
show(fig)

In [59]:
population = Population([
    Portfolio(X=X_test, weights=cvar_model.weights_, name='min CVaR'),
    Portfolio(X=X_test, weights=benchmark.weights_, name='Equal weights'),
    Portfolio(X=X_test, weights=entropic_risk_oce.weights_, name=entropic_risk_oce.risk.name),
    #Portfolio(X=X_test, weights=cvar_risk.weights_, name=cvar_risk.name),
    Portfolio(X=X_test, weights=mono_mv_risk.weights_, name=mono_mv_risk.risk.name),
    Portfolio(X=X_test, weights=quartic_risk.weights_, name=quartic_risk.risk.name)
    ])
population.plot_composition()

In [60]:
fig = population.plot_cumulative_returns()
show(fig)